In [4]:
import pandas as pd
import os
import glob
from datetime import datetime

In [5]:
#setting path to the env variable
folder = os.environ['CAPSTONE_DATA']
folder

'C:\\max\\Google Drive Max\\Max Shared\\Master Data Science\\2020-05 Capstone A\\msds_capstone\\data\\'

# ETL

## Merging original data

In [6]:
path = os.path.join(folder,'original_untidy/Transactions_v2/')
files =  glob.glob(path + "*.xlsx")
dataRaw = pd.DataFrame()
for i,f in enumerate(files):
    print(f)
    dataRaw = dataRaw.append(pd.read_excel(f,skiprows=5),ignore_index=True)

dataRaw=dataRaw.rename(columns={"Chain Master":"Chain Master ID"
                                ,"Unnamed: 2":"Chain Master"
                                ,"Customer":"Customer ID"
                                ,"Unnamed: 4": "Customer"
                                ,"Unnamed: 7":"Vendor ID"
                                ,"Unnamed: 12":"Product ID"
                                ,'Unnamed: 38':'Product Type ID'
                               })
                                
dataRaw.head()


C:\max\Google Drive Max\Max Shared\Master Data Science\2020-05 Capstone A\msds_capstone\data\original_untidy/Transactions_v2\Bivin Report 2013-2015 12.5.19.xlsx
C:\max\Google Drive Max\Max Shared\Master Data Science\2020-05 Capstone A\msds_capstone\data\original_untidy/Transactions_v2\Bivin Report 2016-Nov 2019 12.5.19-Updated.xlsx


,Month,Chain Master ID,Chain Master,Customer ID,Customer,Beverage Type,Vendor,Vendor ID,Brand,Label,...,Product Type,Product Type ID,Qty Per Case,Alcohol Proof,Metrics,STD. Cases,Dollar Sales,9L Cases,Dollar Sales per Case,Dollar Sales per 9L Case
0,Mar-2015,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,FIREBALL,FIREBALL CINN WHSKY,...,FLAVORED,152,12,66.0,NaN,222.0,38734.56,295.26,174.480000,131.187970
1,Dec-2015,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BACARDI USA INC.,1429,GREY GOOSE,GREY GOOSE VODKA,...,VODKA,424,6,80.0,NaN,220.0,39824.40,147.40,181.020000,270.179104
2,Dec-2014,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,FIREBALL,FIREBALL CINN WHSKY,...,FLAVORED,152,12,66.0,NaN,211.0,36815.28,280.63,174.480000,131.187970
3,Dec-2015,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,FIREBALL,FIREBALL CINN WHSKY,...,FLAVORED,152,12,66.0,NaN,202.0,35244.96,268.66,174.480000,131.187970
4,May-2015,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BROWN FORMAN,5,JACK DANIELS TENNESSEE WHISKEY,JACK DANIELS BLK WHSKY,...,WHISKEY,434,12,80.0,NaN,197.0,56483.88,262.01,286.720203,215.579100


## Transformation

In [7]:
# merge iwht STD Cases Table
# https://www.ttb.gov/distilled-spirits/conversion-tables 
cases = pd.DataFrame({"Size":['750 ML', '1.75 LIT / 1.5L', '100 ML', '375 ML', '1.0 LITER','200/187 ML', '50 ML','3 LITER OR LARGER']
                      ,"Size Liters":[.750,1.75 ,.1 ,.375 ,.1 ,.2 ,.05,3]
                      #,"Case Bottles":[12  ,6    ,60 ,24   ,12 ,48 ,120]
                      #,"Case Liters": [9   ,10.50,6  ,9    ,9  ,9.6,6]
                     })


data=pd.merge(dataRaw,cases,how='left',on="Size")

#fix Month
data['Month'] = pd.to_datetime(data['Month'],format="%b-%Y")

#calculate totals
# https://www.inchcalculator.com/convert/case-beer-to-liter-beer/
data['Bottles']=data['Qty Per Case'] * data['STD. Cases']
#data['Liters']=data['Size Liters'] *  data['Bottles'] # conversion is more complex see above link, we need to use the 9l cases unit 
#data['Dollar Sales per Liter']=data['Dollar Sales per 9L Case'] * 9.0  # conversion is more complex see above link, we need to use the 9l cases unit

#adding a 'transactions' column
data['Transactions']=1

#Removing nulls in dollar sales (it happens whnere STD Cases is 0 and there is a credit in Dollar Sales)
data.dropna(subset=['STD. Cases','Dollar Sales per Case','Dollar Sales per 9L Case'],inplace=True)

## Remove negatives!
print(data.shape)
cols=['STD. Cases','Dollar Sales','9L Cases','Dollar Sales per Case','Dollar Sales per 9L Case']
for c in cols: data=data.loc[data[c]>0]
print(data.shape)
print('check for negtives',[sum(data[c]<=0) for c in cols])

data.head()


(235209, 50)
(233152, 50)
check for negtives [0, 0, 0, 0, 0]


,Month,Chain Master ID,Chain Master,Customer ID,Customer,Beverage Type,Vendor,Vendor ID,Brand,Label,...,Alcohol Proof,Metrics,STD. Cases,Dollar Sales,9L Cases,Dollar Sales per Case,Dollar Sales per 9L Case,Size Liters,Bottles,Transactions
0,2015-03-01,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,FIREBALL,FIREBALL CINN WHSKY,...,66.0,NaN,222.0,38734.56,295.26,174.480000,131.187970,0.1,2664.0,1
1,2015-12-01,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BACARDI USA INC.,1429,GREY GOOSE,GREY GOOSE VODKA,...,80.0,NaN,220.0,39824.40,147.40,181.020000,270.179104,0.1,1320.0,1
2,2014-12-01,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,FIREBALL,FIREBALL CINN WHSKY,...,66.0,NaN,211.0,36815.28,280.63,174.480000,131.187970,0.1,2532.0,1
3,2015-12-01,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,FIREBALL,FIREBALL CINN WHSKY,...,66.0,NaN,202.0,35244.96,268.66,174.480000,131.187970,0.1,2424.0,1
4,2015-05-01,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BROWN FORMAN,5,JACK DANIELS TENNESSEE WHISKEY,JACK DANIELS BLK WHSKY,...,80.0,NaN,197.0,56483.88,262.01,286.720203,215.579100,0.1,2364.0,1


# Check nulls

In [8]:
cols=data.columns
data[cols].isna().sum()

Month                               0
Chain Master ID                     0
Chain Master                        0
Customer ID                         0
Customer                            0
Beverage Type                       0
Vendor                              0
Vendor ID                           0
Brand                               0
Label                               0
Size                                0
Product                             0
Product ID                          0
Fiscal Year                         0
Category (CatMan)                   0
Category Rank (CatMan)              0
Product Segment (CatMan)            0
Type (CatMan)                       0
General Flavor (CatMan)             0
Flavor (CatMan)                     0
Age Label (CatMan)             204513
Import or Domestic (CatMan)         0
Country (CatMan)                    0
Region (CatMan)                   581
Origin (CatMan)                     0
Sub Origin (CatMan)                 0
Height (CatM

## Saving

In [9]:
folder = os.environ['CAPSTONE_DATA']
folder
#path = '../../data/tidy_data/Transactions_v2/'
data.to_csv(os.path.join(folder , "Transactions.csv"))
data.to_pickle(os.path.join(folder,"Transactions.pkl"))